<a href="https://colab.research.google.com/github/ChoeTaeBin/Machine-Learnig/blob/main/Banana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##요약

1. quality(좋음/ 나쁨) 분류모델  
장점: 좋은 분류성능의 모델을 만들 수 있을 것 같다.  
단점: 좋고 나쁜것은 주관적인 것으로 의미있는 분류라고 보기 어렵다.  

  
2. 당도 예측 회기모델  
장점: 다른 특성을 가지고 당도를 측정한는 것은 충분히 의미 있는 일인 것 같다.  
  단점: 우선은 성능이 잘 나오지 않는다.  

##데이터에 대한 평가  
 &emsp; &emsp;단위가 무엇인지에 대한 정보가 없다. 가령, 수확시기가 -5라는 것이 무슨 뜻인지 알 수 없다.(스케일링 된 것으로 보임)  

##데이터 셋 선정에서 중요한 기준  
1. 좋은 성능을 낼 수 있으면 좋다.(모델을 만들기 어려우므로 다른 사람이 만든 모델을 찾아보고 결과가 좋은 것을 선택)  
2. 단위가 명확한 것이 추후 발표하기 좋을 것이다.  
3. 의미있는 예측, 분류를 할 수 있어야 한다.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
banana_df = pd.read_csv('/content/drive/MyDrive/banana_quality.csv')
banana_df

,Size,Weight,Sweetness,Softness,HarvestTime,Ripeness,Acidity,Quality
0,-1.924968,0.468078,3.077832,-1.472177,0.294799,2.435570,0.271290,Good
1,-2.409751,0.486870,0.346921,-2.495099,-0.892213,2.067549,0.307325,Good
2,-0.357607,1.483176,1.568452,-2.645145,-0.647267,3.090643,1.427322,Good
3,-0.868524,1.566201,1.889605,-1.273761,-1.006278,1.873001,0.477862,Good
4,0.651825,1.319199,-0.022459,-1.209709,-1.430692,1.078345,2.812442,Good
...,...,...,...,...,...,...,...,...
7995,-6.414403,0.723565,1.134953,2.952763,0.297928,-0.156946,2.398091,Bad
7996,0.851143,-2.217875,-2.812175,0.489249,-1.323410,-2.316883,2.113136,Bad
7997,1.422722,-1.907665,-2.532364,0.964976,-0.562375,-1.834765,0.697361,Bad
7998,-2.131904,-2.742600,-1.008029,2.126946,-0.802632,-3.580266,0.423569,Bad


##banana의 quality(good or bad)를 분륜하는 이진분류 모델  
LogisticRegression이용

In [27]:
banana_input = banana_df.loc[ : , 'Size' : 'Acidity'].to_numpy() #Size ~ Acidity까지 특성으로 사용
banana_input[:5]

array([[-1.9249682 ,  0.46807805,  3.0778325 , -1.4721768 ,  0.2947986 ,
         2.4355695 ,  0.27129033],
       [-2.4097514 ,  0.48686993,  0.34692144, -2.4950993 , -0.8922133 ,
         2.0675488 ,  0.30732512],
       [-0.3576066 ,  1.4831762 ,  1.5684522 , -2.6451454 , -0.64726734,
         3.0906434 ,  1.427322  ],
       [-0.8685235 ,  1.5662014 ,  1.8896049 , -1.2737614 , -1.0062776 ,
         1.8730015 ,  0.47786173],
       [ 0.65182525,  1.3191992 , -0.02245899, -1.2097088 , -1.430692  ,
         1.0783454 ,  2.8124418 ]])

In [28]:
banana_target = np.zeros(len(banana_df['Quality'])) #우선 0으로 초기화
banana_target[banana_df['Quality'] == 'Good'] = 1 #quality가 good이면 1
banana_target

array([1., 1., 1., ..., 0., 0., 0.])

In [29]:
#테스트 세트 분리
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(banana_input, banana_target, random_state = 42)
train_input.shape, test_target.shape

((6000, 7), (2000,))

In [30]:
#정규화
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

In [31]:
#모델 생성, 훈련
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(train_scaled, train_target)

LogisticRegression()

In [33]:
print('<예측>\n', lr.predict(train_scaled[:10]))
print('<실제 답>\n', train_target[:10])

<예측>
 [0. 0. 1. 1. 0. 0. 0. 0. 1. 1.]
<실제 답>
 [0. 0. 1. 1. 0. 0. 0. 0. 1. 1.]


In [35]:
#교차검증

from sklearn.model_selection import cross_validate

scores = cross_validate(lr, train_scaled, train_target, n_jobs = -1)
print('검증점수: ' , np.mean(scores['test_score']))

검증점수:  0.8776666666666667


In [48]:
#최종평가
print('테스트 점수: ', lr.score(test_scaled, test_target))

테스트 점수:  0.879


##banana의 Sweetness(당도)를 예측하는 회기 모델  
Ridge(릿지 회기)이용

In [114]:
#Sweetness, Quality를 제외한 나머지 특성을 input으로 사용
banana_input2  = np.array(banana_df[['Size', 'Weight', 'Softness', 'HarvestTime',	'Ripeness', 'Acidity']])

In [115]:
#Sweetness를 target으로 함
banana_target2 = np.array(banana_df['Sweetness'])

In [116]:
#다항항들을 추가함
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures()
poly.fit(banana_input2)
input_poly = poly.transform(banana_input2)
input_poly.shape

(8000, 28)

In [117]:
#Sweetness를 target으로 함
banana_target2 = np.array(banana_df['Sweetness'])

In [118]:
#테스트 세트 분리
train_input2, test_input2, train_target2, test_target2 = train_test_split(input_poly, banana_target2, random_state = 42)
train_input2.shape, test_target2.shape

((6000, 28), (2000,))

In [119]:
#정규화
ss = StandardScaler()
ss.fit(train_input2)
train_scaled2 = ss.transform(train_input2)
test_scaled2 = ss.transform(test_input2)

In [127]:
#모델 생성 훈련
from sklearn.linear_model import Ridge

ridge = Ridge()
ridge.fit(train_scaled2, train_target2)

Ridge()

In [128]:
print('<예측>\n', ridge.predict(train_scaled2[:10]))
print('<실제 답>\n', train_target2[:10])

<예측>
 [ 0.05985419 -1.21944089 -1.08894468  1.85542356 -1.75900393 -1.67955309
 -0.77083324 -0.91990425 -3.91353967  0.95193675]
<실제 답>
 [ 0.45254257 -2.7922335  -1.641051    1.0845554  -2.1179473  -1.2542796
 -1.1625501  -2.2108197  -2.6741517  -0.79300845]


In [129]:
#최종평가
print('훈련세트 점수: ', ridge.score(train_scaled2, train_target2))
print('테스트 점수: ', ridge.score(test_scaled2, test_target2))

훈련세트 점수:  0.4879395011155311
테스트 점수:  0.5144856638606538
